<a href="https://colab.research.google.com/github/adamstiefel/AI-Business-Agents/blob/main/Process_Mapping_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI-Powered Process Mapping Agent 🚀

**Transforming Process Documentation into Actionable Insights**

## What This Tool Does

This Google Colab notebook implements an **AI-Powered Process Mapping Agent**. It takes your existing process documentation – whether it's in text format, extracted from PDFs, or copied from platforms like Confluence – and uses a Large Language Model (LLM) to automatically:

1.  **Understand** the described process flow, steps, decisions, and sequences.
2.  **Generate** a structured representation of that process in a diagramming language (like Mermaid.js or PlantUML).
3.  **Visualize** the process as a clear, easy-to-understand flowchart.

Essentially, it automates the often time-consuming task of creating initial process diagrams from written materials, allowing for rapid visualization and analysis.

## How This Delivers Business Value (The "Makes Money" Aspect)

This tool isn't a direct revenue generator, but it provides significant business value that translates into cost savings, efficiency gains, improved outcomes, and ultimately, enhanced profitability. Here's how:

* **💰 Drastically Reduce Manual Effort & Costs:**
    * **Save Billable Hours:** Automating diagram creation frees up valuable time for analysts, consultants, and subject matter experts who would otherwise spend hours manually drawing flowcharts. This directly translates to reduced project costs or allows resources to focus on higher-value strategic analysis.
    * **Accelerate Project Delivery:** Quickly generate initial process maps to kickstart analysis, workshops, and improvement initiatives, leading to faster project completion and quicker realization of benefits.

* **📈 Boost Operational Efficiency:**
    * **Rapid Process Visualization:** Quickly convert lengthy documents into clear visual maps, making it easier to identify bottlenecks, redundancies, and areas for optimization within your operations.
    * **Standardized Understanding:** Ensure everyone views and understands processes consistently, reducing misinterpretations and errors that can cost time and money.

* **💡 Enhance Communication & Collaboration:**
    * **Clearer Insights:** Visual diagrams are far more intuitive and easier to discuss than text-heavy documents, leading to more productive conversations among stakeholders (business

In [ ]:
#@title 1. Setup and Installations
# --- Install necessary libraries ---
!pip install openai PyPDF2 -q

import openai
import PyPDF2
import io
import base64
import zlib
import requests
from IPython.display import display, Markdown, Image, HTML
from google.colab import files, userdata # For API key and file uploads

print("Libraries installed and imported.")

Libraries installed and imported.


In [ ]:
#@title 2. OpenAI API Key Configuration
# --- Configure your OpenAI API Key ---
# Recommended: Use Colab Secrets (Key icon on the left sidebar)
# Add a secret named 'OPENAI_API_KEY' with your actual OpenAI API key.

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if OPENAI_API_KEY:
        openai.api_key = OPENAI_API_KEY
        print("OpenAI API Key loaded successfully from Colab Secrets.")
    else:
        print("WARNING: OpenAI API Key not found in Colab Secrets.")
        print("Please add it or paste it directly into the cell below (less secure).")
        # Fallback: allow pasting key directly if not in secrets (for quick tests)
        # OPENAI_API_KEY = "YOUR_API_KEY_HERE" # PASTE YOUR KEY HERE IF NOT USING SECRETS
        # if OPENAI_API_KEY == "YOUR_API_KEY_HERE":
        #     print("Please replace 'YOUR_API_KEY_HERE' with your actual key.")
        # else:
        #     openai.api_key = OPENAI_API_KEY
        #     print("OpenAI API Key set directly.")
except Exception as e:
    print(f"Error accessing Colab Secrets: {e}")
    print("Please ensure you have an OpenAI API key.")
    OPENAI_API_KEY = None # Ensure it's None if there's an error or no key

# You can manually set it here if not using secrets, but it's less secure:
# if not OPENAI_API_KEY:
#   OPENAI_API_KEY = "sk-YOUR_OPENAI_API_KEY_HERE" # <-- PASTE YOUR KEY HERE
#   if "sk-YOUR_OPENAI_API_KEY_HERE" == OPENAI_API_KEY:
#       print("🛑 API Key not set. Please replace 'sk-YOUR_OPENAI_API_KEY_HERE' with your actual key to proceed.")
#   else:
#       openai.api_key = OPENAI_API_KEY
#       print("✅ OpenAI API Key set directly (less secure).")

if not openai.api_key:
    print("🛑 OpenAI API Key is NOT SET. LLM functionality will not work.")
    print("Please add your key to Colab Secrets (recommended) or set it manually in this cell.")

OpenAI API Key loaded successfully from Colab Secrets.


In [ ]:
#@title 3. Input Process Documentation

#@markdown Choose your input method:
input_method = "Text Area" #@param ["Text Area", "PDF Upload", "Sample Text"]

#@markdown ---
#@markdown ### Option A: Paste Text Directly
#@markdown (If you chose "Text Area" above, paste your process description here)
process_text_area = """
Start Process: New Employee Onboarding

1. HR receives signed offer letter.
2. HR initiates background check.
3. If background check is clear:
    a. HR sends welcome email with start date and first-day instructions.
    b. IT is notified to prepare equipment (laptop, accounts).
    c. Manager is notified to prepare an onboarding plan.
4. If background check has issues:
    a. HR reviews issues with legal.
    b. Decision: Proceed or Rescind Offer.
    c. If Rescind Offer, HR communicates to candidate. Process Ends.
    d. If Proceed, continue to step 3a.
5. On first day, employee meets with HR for paperwork and orientation.
6. Employee meets with manager for team introduction and role overview.
7. IT provides equipment and system access.
8. Employee begins onboarding tasks as per manager's plan.

End Process.
""" #@param {type:"string"}

#@markdown ---
#@markdown ### Option B: Upload PDF
#@markdown (If you chose "PDF Upload", a file dialog will appear when you run the cell below this section)

#@markdown ---
#@markdown ### Option C: Use Sample Text
#@markdown (If you chose "Sample Text", the predefined sample below will be used)
sample_process_text = """
Start: Customer Order Process
1. Customer places an order online.
2. System checks inventory.
3. If item is in stock:
    a. Payment is processed.
    b. If payment successful:
        i. Order confirmation sent to customer.
        ii. Warehouse is notified to pack and ship the item.
        iii. Item shipped.
        iv. Tracking information sent to customer.
    c. Else (payment failed):
        i. Customer notified of payment failure.
        ii. Order on hold.
4. Else (item not in stock):
    a. Customer notified of backorder status.
    b. Option given to wait or cancel.
End
"""

# --- Variables to store the final process input ---
final_process_input = ""
pdf_text_content = "" # To store extracted PDF text

def extract_text_from_pdf(file_bytes):
    """Extracts text from an uploaded PDF file."""
    text = ""
    try:
        with io.BytesIO(file_bytes) as open_pdf_file:
            reader = PyPDF2.PdfReader(open_pdf_file)
            if reader.is_encrypted:
                try:
                    reader.decrypt('') # Try with an empty password
                except Exception as e:
                    print(f"Could not decrypt PDF: {e}. Text extraction might fail or be incomplete.")

            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text()
    except Exception as e:
        return f"Error reading PDF: {e}"
    return text

if input_method == "Text Area":
    final_process_input = process_text_area
    print("Using text from Text Area.")
elif input_method == "Sample Text":
    final_process_input = sample_process_text
    print("Using sample process text.")
elif input_method == "PDF Upload":
    print("Please upload a PDF file using the button that will appear below.")
    # This cell needs to be run, then the next cell for PDF upload.
    # We'll handle the actual upload in the "Process and Generate" step to ensure order.
    pass # PDF processing will happen in the main execution cell

print("\n--- Process Input Preview (first 500 chars) ---")
if final_process_input:
    print(final_process_input[:500] + "...")
elif input_method == "PDF Upload":
    print("PDF will be processed in the next step if 'PDF Upload' is selected.")
else:
    print("No input provided yet.")

Using text from Text Area.

--- Process Input Preview (first 500 chars) ---

Start Process: New Employee Onboarding

1. HR receives signed offer letter.
2. HR initiates background check.
3. If background check is clear:
    a. HR sends welcome email with start date and first-day instructions.
    b. IT is notified to prepare equipment (laptop, accounts).
    c. Manager is notified to prepare an onboarding plan.
4. If background check has issues:
    a. HR reviews issues with legal.
    b. Decision: Proceed or Rescind Offer.
    c. If Rescind Offer, HR communicates to ca...


In [ ]:
#@title 4. LLM Interaction and Diagram Code Generation
#@markdown Select the diagram type you want to generate:
diagram_type = "Mermaid.js" #@param ["Mermaid.js", "PlantUML"]
#@markdown Select the LLM model (GPT-4 Turbo is recommended for complex diagrams if you have API access):
llm_model = "gpt-4-turbo-preview" #@param ["gpt-4-turbo-preview", "gpt-4", "gpt-3.5-turbo"]

# Ensure necessary imports - these are typically needed for display functions
from IPython.display import display, Markdown, HTML, Image
import base64 # For encoding for mermaid.ink and plantuml
import zlib   # For plantuml
# import requests # Not directly needed here if using Image(url=...) but often used with APIs

def generate_diagram_code_with_llm(text_description, target_diagram_type="mermaid", model=llm_model):
    """
    Uses an LLM to generate diagram code (Mermaid or PlantUML) from text,
    with enhanced presentation guidelines for Mermaid.js.
    """
    if not openai.api_key:
        return "Error: OpenAI API key not set. Please configure it in Step 2."
    if not text_description or text_description.strip() == "":
        return "Error: Input process description is empty."

    if target_diagram_type == "Mermaid.js":
        prompt = f"""
        You are an expert process analyst and a specialist in creating highly presentable and logically accurate Mermaid.js flowchart diagrams.
        Convert the following process description into a Mermaid.js flowchart diagram.

        Follow these guidelines for a presentable and correct diagram:
        1. Use 'graph TD' for a top-down flowchart.
        2. Start and End Events: Use rounded rectangles. For example: `A_start[(Start Process)]`, `Z_end[(End Process)]`.
        3. Tasks/Activities: Use standard rectangles. For example: `B_task1[Describe the task here]`.
        4. Decision Points: Use diamond shapes. For example: `C_decision{{Is condition met?}}`.
        5. Decision Flows: Clearly label the links exiting decision diamonds (e.g., `C_decision -- Yes --> D_task_yes`, `C_decision -- No --> E_task_no`).
        6. Node IDs: Use reasonably short and descriptive IDs (e.g., `step1`, `decisionA`). The text within the brackets/shapes is what will be displayed.
        7. Clarity: Ensure the flow is logical and easy to follow. All steps from the input must be accurately represented.
        8. Conciseness: Keep text within nodes clear and concise.
        9. Connections: Ensure all nodes are appropriately connected to reflect the true sequence and dependencies of the process.
        10. Parallelism & Dependencies: If the process describes activities that can happen in parallel (e.g., 'system notifies X' and 'system notifies Y' after a single event), represent these as distinct parallel branches stemming from the common preceding step. Ensure that any subsequent steps dependent on the completion of these parallel activities correctly show these dependencies (e.g., by joining the parallel branches with arrows pointing to the dependent step).
        11. Termination: Ensure ALL process paths properly terminate at a defined End event node. If there are multiple distinct outcomes leading to an end, each should clearly connect to an appropriate End event.

        Process Description:
        ---
        {text_description}
        ---

        Mermaid.js Code (ensure it starts with ```mermaid and ends with ``` and contains ONLY the Mermaid code):
        """
    elif target_diagram_type == "PlantUML":
        prompt = f"""
        You are an expert process analyst and a specialist in creating PlantUML diagrams.
        Convert the following process description into a PlantUML activity diagram.
        The diagram should clearly represent the sequence of steps, decision points (using 'if/else/endif' or 'switch/case'), forks/joins for parallel activities (if any), and start/stop points.
        Use standard PlantUML activity diagram syntax.
        Label decision paths clearly.
        Ensure all steps mentioned in the input are covered in the diagram.
        Output ONLY the PlantUML code, starting with '@startuml' and ending with '@enduml'.
        Do NOT include any other explanatory text before or after the PlantUML code block.

        Process Description:
        ---
        {text_description}
        ---

        PlantUML Code:
        """
    else:
        return "Error: Unsupported diagram type. Choose 'Mermaid.js' or 'PlantUML'."

    print(f"\n🤖 Sending request to LLM ({model}) to generate {target_diagram_type} code with enhanced presentation guidelines...")
    print("This might take a few moments...")

    try:
        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an assistant that generates diagram code from process descriptions following specific presentation and logical accuracy guidelines."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
            max_tokens=2000
        )
        generated_code = response.choices[0].message.content.strip()

        if target_diagram_type == "Mermaid.js":
            if "```mermaid" in generated_code:
                code_match = generated_code.split("```mermaid")
                if len(code_match) > 1:
                    generated_code = code_match[1].split("```")[0].strip()
                else:
                    generated_code = generated_code.replace("```mermaid", "").replace("```", "").strip()
            if not generated_code.lower().startswith("graph"):
                 return f"Error: LLM did not return valid Mermaid.js code starting with 'graph'. It returned:\n{generated_code}"
        elif target_diagram_type == "PlantUML":
            if "@startuml" not in generated_code:
                 generated_code = "@startuml\n" + generated_code
            if "@enduml" not in generated_code:
                 generated_code = generated_code + "\n@enduml"
            if "```plantuml" in generated_code:
                generated_code = generated_code.split("```plantuml")[1].split("```")[0].strip()
            elif "```" in generated_code and "@startuml" in generated_code:
                 generated_code = generated_code.split("```")[1].strip()
                 if generated_code.startswith("plantuml"):
                    generated_code = generated_code.split("\n",1)[1]

        print(f"✅ LLM generated {target_diagram_type} code successfully.")
        return generated_code
    except openai.APIError as e:
        return f"OpenAI API Error: {e}"
    except Exception as e:
        return f"Error during LLM call: {e}"

# --- Functions for rendering ---

def plantuml_deflate_and_encode(plantuml_text):
    """Deflate and encode PlantUML text for URL generation (PlantUML server)."""
    plantuml_text_bytes = plantuml_text.encode('utf-8')
    compressor = zlib.compressobj(level=9, method=zlib.DEFLATED, wbits=-15)
    compressed_data = compressor.compress(plantuml_text_bytes) + compressor.flush()
    encoded_string = base64.urlsafe_b64encode(compressed_data).decode('utf-8')
    return encoded_string

def display_plantuml(plantuml_code):
    """Displays PlantUML code as an image using the online PlantUML server."""
    if not plantuml_code.strip().startswith("@startuml"):
        plantuml_code = "@startuml\n" + plantuml_code
    if not plantuml_code.strip().endswith("@enduml"):
        plantuml_code = plantuml_code + "\n@enduml"
    encoded_plantuml_data = plantuml_deflate_and_encode(plantuml_code)
    prefixed_encoded_plantuml = "~1" + encoded_plantuml_data
    plantuml_server_url = "http://www.plantuml.com/plantuml/png/"
    image_url = plantuml_server_url + prefixed_encoded_plantuml
    print(f"PlantUML Image URL (with ~1 prefix): {image_url}")
    try:
        display(HTML(f'<img src="{image_url}" alt="PlantUML Diagram" style="max-width: 100%; height: auto; border:1px solid #ccc;">'))
    except Exception as e:
        print(f"Could not display PlantUML image directly from URL: {e}")
        display(Markdown(f"You can view the PlantUML diagram by pasting the code below into [PlantText](https://www.planttext.com/) or a similar online renderer, or by visiting the [generated image URL]({image_url})."))
        display(Markdown(f"```plantuml\n{plantuml_code}\n```"))

# --- Main Execution for this cell (Process and Generate) ---

if 'input_method' not in globals():
    print("🛑 'input_method' not defined. Please run Cell 3 (Input Process Documentation) first.")
    final_process_input = ""
elif input_method == "PDF Upload":
    if 'extract_text_from_pdf' not in globals(): # Assumes extract_text_from_pdf is defined in an earlier cell (e.g. Cell 3)
        print("🛑 PDF helper function 'extract_text_from_pdf' not found. Ensure Cell 3's definitions are run.")
        final_process_input = ""
    else:
        print("Please upload your PDF file:")
        try:
            uploaded_files = files.upload()
        except NameError:
            print("🛑 'files' (for upload) not defined. Make sure 'from google.colab import files' has been run.")
            uploaded_files = {}

        if not uploaded_files:
            print("No file uploaded. Please upload a PDF or choose another input method.")
            final_process_input = ""
        else:
            for fn in uploaded_files.keys():
                print(f'\nProcessing uploaded file: "{fn}" ({len(uploaded_files[fn])} bytes)')
                pdf_text_content_temp = extract_text_from_pdf(uploaded_files[fn])
                if "Error reading PDF" in pdf_text_content_temp:
                    print(f"⚠️ {pdf_text_content_temp}")
                    final_process_input = ""
                else:
                    final_process_input = pdf_text_content_temp
                    print(f"✅ Text extracted from PDF successfully.")
                    print("\n--- Extracted PDF Text Preview (first 500 chars) ---")
                    print(final_process_input[:500] + "...")
                break

if 'final_process_input' not in globals() or not final_process_input or final_process_input.strip() == "":
    display(Markdown("<hr>"))
    print("🛑 No process input provided or extracted. Cannot generate diagram.")
    print("Please select an input method and provide data in Step 3, then re-run this cell (after running Step 3).")
elif not openai.api_key:
    display(Markdown("<hr>"))
    print("🛑 OpenAI API Key is not set. Cannot generate diagram.")
    print("Please configure your API key in Step 2 and re-run this cell.")
else:
    generated_code = generate_diagram_code_with_llm(final_process_input, diagram_type, llm_model)
    display(Markdown("<hr>"))

    if generated_code.startswith("Error:"):
        print(f"⚠️ {generated_code}")
    else:
        print(f"\n✨ Generated {diagram_type} Code:\n")
        # Display the code itself using Markdown code block for easy copying
        if diagram_type == "Mermaid.js":
            display(Markdown(f"```mermaid\n{generated_code}\n```"))
        elif diagram_type == "PlantUML":
            display(Markdown(f"```plantuml\n{generated_code}\n```"))

        print(f"\n--- Attempting to Render {diagram_type} Diagram ---")

        if diagram_type == "Mermaid.js":
            # Method 1: Native Colab Markdown Rendering (Ideal)
            print("\nAttempt 1: Native Colab Rendering (should appear directly above the 'Mermaid.js Diagram' title after the code block).")
            # This was already done by displaying the markdown code block above.
            # We add this print statement as a guide.

            # Method 2: Fallback to External Rendering Service (mermaid.ink) as an image
            print("\nAttempt 2: Rendering as an image via mermaid.ink (if native rendering fails or for a guaranteed visual).")
            try:
                mermaid_code_bytes = generated_code.encode('utf-8')
                base64_bytes = base64.urlsafe_b64encode(mermaid_code_bytes) # Use urlsafe_b64encode
                base64_string = base64_bytes.decode('utf-8')

                image_url_mermaid_ink = f"https://mermaid.ink/img/{base64_string}"
                # For SVG: image_url_mermaid_ink = f"https://mermaid.ink/svg/{base64_string}" # SVG often looks better

                print(f"Mermaid.ink Image URL: {image_url_mermaid_ink}")
                display(Image(url=image_url_mermaid_ink))
                print("--- Mermaid.js Diagram from mermaid.ink (rendered as image above) ---")
            except Exception as e:
                print(f"⚠️ Could not render image using mermaid.ink: {e}")

            print("\n🔎 If NO diagram is rendering visually:")
            print("   1. Ensure your browser allows images from external sites (mermaid.ink).")
            print("   2. Try re-running this cell. Sometimes Colab/browsers need a refresh.")
            print("   3. Try a hard refresh of your browser page (Ctrl+Shift+R or Cmd+Shift+R).")
            print("   4. Try a different web browser.")
            print("   5. Check your browser's JavaScript console (F12 -> Console) for errors.")
            print(  "   6. You've confirmed the code works on mermaid.live, which is great! This means the code is valid.")

        elif diagram_type == "PlantUML":
            display_plantuml(generated_code)


🤖 Sending request to LLM (gpt-4-turbo-preview) to generate Mermaid.js code with enhanced presentation guidelines...
This might take a few moments...
✅ LLM generated Mermaid.js code successfully.


<hr>


✨ Generated Mermaid.js Code:



```mermaid
graph TD
    A_start[(Start Process: New Employee Onboarding)]
    B_receive[HR receives signed offer letter]
    C_initiate[HR initiates background check]
    D_decision{Background check clear?}
    E_clear[HR sends welcome email with start date and first-day instructions]
    F_IT_notify[IT is notified to prepare equipment]
    G_manager_notify[Manager is notified to prepare an onboarding plan]
    H_issues[HR reviews issues with legal]
    I_decision{Proceed or Rescind Offer?}
    J_rescind[HR communicates to candidate]
    K_end1[(End Process)]
    L_first_day[On first day, employee meets with HR for paperwork and orientation]
    M_manager_meeting[Employee meets with manager for team introduction and role overview]
    N_IT_provides[IT provides equipment and system access]
    O_onboarding_tasks[Employee begins onboarding tasks as per manager's plan]
    P_end2[(End Process)]

    A_start --> B_receive --> C_initiate --> D_decision
    D_decision -- Yes --> E_clear --> F_IT_notify & G_manager_notify --> L_first_day
    D_decision -- No --> H_issues --> I_decision
    I_decision -- Rescind Offer --> J_rescind --> K_end1
    I_decision -- Proceed --> E_clear
    L_first_day --> M_manager_meeting --> N_IT_provides --> O_onboarding_tasks --> P_end2
```


--- Attempting to Render Mermaid.js Diagram ---

Attempt 1: Native Colab Rendering (should appear directly above the 'Mermaid.js Diagram' title after the code block).

Attempt 2: Rendering as an image via mermaid.ink (if native rendering fails or for a guaranteed visual).
Mermaid.ink Image URL: https://mermaid.ink/img/Z3JhcGggVEQKICAgIEFfc3RhcnRbKFN0YXJ0IFByb2Nlc3M6IE5ldyBFbXBsb3llZSBPbmJvYXJkaW5nKV0KICAgIEJfcmVjZWl2ZVtIUiByZWNlaXZlcyBzaWduZWQgb2ZmZXIgbGV0dGVyXQogICAgQ19pbml0aWF0ZVtIUiBpbml0aWF0ZXMgYmFja2dyb3VuZCBjaGVja10KICAgIERfZGVjaXNpb257QmFja2dyb3VuZCBjaGVjayBjbGVhcj99CiAgICBFX2NsZWFyW0hSIHNlbmRzIHdlbGNvbWUgZW1haWwgd2l0aCBzdGFydCBkYXRlIGFuZCBmaXJzdC1kYXkgaW5zdHJ1Y3Rpb25zXQogICAgRl9JVF9ub3RpZnlbSVQgaXMgbm90aWZpZWQgdG8gcHJlcGFyZSBlcXVpcG1lbnRdCiAgICBHX21hbmFnZXJfbm90aWZ5W01hbmFnZXIgaXMgbm90aWZpZWQgdG8gcHJlcGFyZSBhbiBvbmJvYXJkaW5nIHBsYW5dCiAgICBIX2lzc3Vlc1tIUiByZXZpZXdzIGlzc3VlcyB3aXRoIGxlZ2FsXQogICAgSV9kZWNpc2lvbntQcm9jZWVkIG9yIFJlc2NpbmQgT2ZmZXI_fQogICAgSl9yZXNjaW5kW0hSIGNvbW11bml

--- Mermaid.js Diagram from mermaid.ink (rendered as image above) ---

🔎 If NO diagram is rendering visually:
   1. Ensure your browser allows images from external sites (mermaid.ink).
   2. Try re-running this cell. Sometimes Colab/browsers need a refresh.
   3. Try a hard refresh of your browser page (Ctrl+Shift+R or Cmd+Shift+R).
   4. Try a different web browser.
   5. Check your browser's JavaScript console (F12 -> Console) for errors.
   6. You've confirmed the code works on mermaid.live, which is great! This means the code is valid.
